# Group 5 - Online Gambling Analysis

#### Théau Bruno | Alexandra-Maria Ionascu | Venkat Jayanarasimhan

In [2]:
# Installing required packages

if(!require("tidyr")) install.packages("tidyr"); library("tidyr")
if(!require("plyr")) install.packages("plyr"); library("plyr")
if(!require("dplyr")) install.packages("dplyr"); library("dplyr")
if(!require("lubridate")) install.packages("lubridate"); library("lubridate")
if(!require("scales")) install.packages("scales"); library("scales")
if(!require("dplyr")) install.packages("tidyverse"); 
if(!require("reshape2")) install.packages("reshape2"); 
library(haven)
library(dplyr)
library(readr)
library(tidyverse)
library(reshape2)

In [3]:
# Loading Country Dataset

countries <- read.csv("Country.csv")
head(countries)

Country,CountryName,lon,lat
4,Afghanistan,67.709953,33.93911
8,Albania,20.168331,41.15333
10,Antarctica,135.000000,-82.86275
12,Algeria,1.659626,28.03389
16,American Samoa,-170.132217,-14.27097
20,Andorra,1.521801,42.50628


In [4]:
# Reading Demographics Dataset

demo <- read_sas("RawDataIDemographics.sas7bdat")
demo$Gender <- mapvalues(demo$Gender, from=c(1,0), to=c('male', 'female'))
demo$Language <- demo$Language <- mapvalues(demo$Language,from=c(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22), to=c("English","German","Italian","Spanish","Swedish","French","Turkish","Greek","Polish","Norwegian","Danish","Catalan",
     "Czech","Hungarian","Dutch","Portugues","Russian","Slovenian","Croatian","Slovak","Simple Chinese","Traditional Chinese"))
demo$FirstPay <- ymd(demo$FirstPay)
demo$FirstAct <- ymd(demo$FirstAct)
demo$FirstSp <- ymd(demo$FirstSp)

demo$FirstCa <- ymd(demo$FirstCa)
demo$FirstGa <- ymd(demo$FirstGa)
demo$FirstPo <- ymd(demo$FirstPo)

demo <- dplyr::rename(demo, RegistrationDate = RegDate)
demo$RegistrationDate <- ymd(demo$RegistrationDate)



demo$ApplicationID <- mapvalues(demo$ApplicationID, from=c(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22,23,24,26,27,28,29,30,31,32,33,36,38,42),
to=c("BETANDWIN.COM","TRIPLE-A-CASINO.COM","BETANDWIN.DE","WAP.BETANDWIN.COM","SMS MOBILE BETTING APPLICATION","CHARITY",
"DOLCEVITACASINO.COM","BALLS OF FIRE","BETEUROPE.COM","BAHSEGIR.COM","CASINO.BETEUROPE.COM","WWW.CASINOTURK.COM","WWW.SANALCASINO.COM",
"BETOTO.COM","PLAYIT.COM","CASINO.PLAYIT.COM","HECROUPIER.COM","SMS.BETANDWIN.COM","WAP.BETANDWIN.DE","BOF.PLAYIT.COM","BOF.BETEUROPE.COM",
"BETANDWIN POKER","BETANDWIN CASINO","SMS.BETANDWIN.DE","LOTTERY.BETOTO.COM",
"BETWORK.COM","WAP.PLAYIT.COM","PLAYIT POKER","BETEUROPE POKER","BETOTO POKER","BETANDWIN GAMES","BETOTO CASINO","BETEUROPE GAMES","PLAYIT GAMES"))

demo <- dplyr::inner_join(demo, countries, by="Country")

head(demo)

length(unique(demo$UserID))
length(demo$UserID)

The following `from` values were not present in `x`: 18, 19, 20, 21, 22
Warning message:
" 1236 failed to parse."Warning message:
" 36761 failed to parse."Warning message:
" 39376 failed to parse."Warning message:
" 40507 failed to parse."The following `from` values were not present in `x`: 2, 5, 6, 7, 10, 12, 13, 17, 18, 26, 28, 29


UserID,Country,Language,RegistrationDate,FirstPay,FirstAct,FirstSp,FirstCa,FirstGa,FirstPo,ApplicationID,Gender,CountryName,lon,lat
1324354,276,German,2005-02-01,2005-02-24,2005-02-24,2005-02-24,NA,NA,NA,BETANDWIN.DE,male,Germany,10.45153,51.16569
1324355,300,Greek,2005-02-01,2005-02-01,2005-02-01,2005-02-01,NA,NA,2005-06-11,BETANDWIN.COM,male,Greece,21.82431,39.07421
1324356,276,German,2005-02-01,2005-02-01,2005-02-02,2005-02-02,NA,NA,NA,BETANDWIN.DE,male,Germany,10.45153,51.16569
1324358,752,English,2005-02-01,2005-02-01,2005-02-01,2005-02-01,NA,NA,NA,BETANDWIN.COM,male,Sweden,18.64350,60.12816
1324360,792,Turkish,2005-02-01,2005-02-02,2005-02-02,2005-02-02,2005-02-03,NA,NA,BETEUROPE.COM,male,Turkey,35.24332,38.96375
1324362,276,German,2005-02-01,2005-02-11,2005-02-11,2005-02-11,NA,NA,NA,BETANDWIN.DE,male,Germany,10.45153,51.16569


[1] 42649

[1] 42649

In [5]:
# Checking column names

colnames(demo)

[1] "UserID"           "Country"          "Language"         "RegistrationDate"
 [5] "FirstPay"         "FirstAct"         "FirstSp"          "FirstCa"         
 [9] "FirstGa"          "FirstPo"          "ApplicationID"    "Gender"          
[13] "CountryName"      "lon"              "lat"

In [6]:
# Reading Poker Chip Conversions

poker <- read_sas("RawDataIIIPokerChipConversions.sas7bdat")
poker$TransType <- mapvalues(poker$TransType, from=c(124,24), to=c('POwinnings', 'PObets'))
poker$TransDateTime <- as.POSIXct(poker$TransDateTime, format='%Y-%m-%d')

poker <- poker %>% filter(TransDateTime <= ymd(20050930) )

poker_dates <- poker %>% select(-TransType, -TransAmount) %>% 
group_by(UserID) %>% 
transmute(FirstPo = first(TransDateTime), LastPo = last(TransDateTime), PObets = n_distinct(TransDateTime)) %>% 
distinct(UserID, .keep_all = TRUE)

poker_win <- poker %>%  
filter(TransType == 'POwinnings') %>% 
group_by(UserID) %>% 
transmute(POwin = sum(TransAmount)) %>%
distinct(UserID, .keep_all = TRUE)

poker_bet <- poker %>%  
filter(TransType == 'PObets') %>% 
group_by(UserID) %>% 
transmute(POstakes = sum(TransAmount)) %>%
distinct(UserID, .keep_all = TRUE)


poker_merged <- dplyr::full_join(poker_dates, poker_win, by="UserID")
poker_merged <- dplyr::full_join(poker_merged, poker_bet, by="UserID")
poker_merged[(is.na(poker_merged))] <-0

length(unique(poker_merged$UserID))
length(poker_merged$UserID)
head(poker_merged)

[1] 2369

[1] 2369

UserID,FirstPo,LastPo,PObets,POwin,POstakes
1324355,2005-06-12,2005-06-15,3,30.8295000,8.2593000
1324368,2005-04-05,2005-05-30,23,9161.6582000,9620.8147000
1324369,2005-03-03,2005-03-03,1,0.2949455,0.2949455
1324371,2005-02-01,2005-02-10,4,122.6326185,65.3674456
1324372,2005-09-02,2005-09-30,26,786.2004470,848.4530022
1324377,2005-02-01,2005-02-03,2,27.5521477,12.9949062


In [7]:
# Reading Poker Chip Conversions

POmonth <- read_sas("RawDataIIIPokerChipConversions.sas7bdat")
POmonth$TransDateTime <- as.POSIXct(POmonth$TransDateTime, format='%Y-%m-%d')
POmonth$month <- month(as.POSIXlt(POmonth$TransDateTime, format="%Y/%m/%d"))
POmonth$TransType <- mapvalues(POmonth$TransType, from=c(124,24), to=c('win', 'stakes'))

POmonth <- POmonth %>% group_by(UserID)%>% filter(TransType == "stakes") %>% select(-TransDateTime,-TransType) %>% pivot_wider(names_from =month,  values_from =c(TransAmount), values_fn = {sum})
POmonth[is.na(POmonth)] <- 0 
colnames(POmonth) <- gsub(" ", "",paste("po_", colnames(POmonth)))
POmonth <- rename(POmonth,c('UserID'='po_UserID'))


allmonth <- read_sas("RawDataIIUserDailyAggregation.sas7bdat")
allmonth$Date <- ymd(allmonth$Date)
allmonth$month <- month(as.POSIXlt(allmonth$Date, format="%Y/%m/%d"))

allmonth <- allmonth %>% group_by(UserID) %>% select(-Bets, -ProductID, -Winnings, -Date) %>% pivot_wider(names_from =month,  values_from =c(Stakes), values_fn = {sum})
allmonth[is.na(allmonth)] <- 0 
colnames(allmonth) <- gsub(" ", "",paste("all_", colnames(allmonth)))
allmonth <- rename(allmonth,c('UserID'='all_UserID'))

head(allmonth)

head(POmonth)

UserID,all_2,all_3,all_4,all_5,all_6,all_7,all_8,all_9
1324354,40.0000,1814.1400,3781.63,2975.5900,80.00,0,1908.28,1376.9700
1324355,81.0600,66.9400,52.92,17.1500,51.96,0,50.00,105.5300
1324356,116.9600,182.4100,0.50,0.0000,0.00,0,639.34,426.0500
1324358,120.7323,0.0000,0.00,215.5575,0.00,0,0.00,0.0000
1324360,21.8407,5.7226,0.00,0.0000,0.00,0,0.00,38.1794
1324362,10.0000,0.0000,0.00,0.0000,0.00,0,0.00,12.0000


UserID,po_6,po_4,po_5,po_3,po_2,po_9,po_10,po_7,po_8
1324355,8.2593,0.000,0.000,0.0000000,0.00000,0.000,0.00000,0,0
1324368,0.0000,5035.346,4585.469,0.0000000,0.00000,0.000,0.00000,0,0
1324369,0.0000,0.000,0.000,0.2949455,0.00000,0.000,0.00000,0,0
1324371,0.0000,0.000,0.000,0.0000000,65.36745,0.000,0.00000,0,0
1324372,0.0000,0.000,0.000,0.0000000,0.00000,848.453,69.96067,0,0
1324377,0.0000,0.000,0.000,0.0000000,12.99491,0.000,0.00000,0,0


In [8]:
# Calculating Total Stakes Per Month

totalstakesmonth <- dplyr::full_join(allmonth, POmonth, by='UserID')
totalstakesmonth[is.na(totalstakesmonth)] <- 0 
totalstakesmonth <- totalstakesmonth %>% group_by(UserID) %>% transmute(February= all_2+po_2, March=all_3+po_3, April = all_4+po_4, May= all_5 +po_5, June=all_6+po_6, July=all_7+po_7, August=all_8+po_8, September=all_9+po_9)
head(totalstakesmonth)

UserID,February,March,April,May,June,July,August,September
1324354,40.0000,1814.1400,3781.63,2975.5900,80.0000,0,1908.28,1376.9700
1324355,81.0600,66.9400,52.92,17.1500,60.2193,0,50.00,105.5300
1324356,116.9600,182.4100,0.50,0.0000,0.0000,0,639.34,426.0500
1324358,120.7323,0.0000,0.00,215.5575,0.0000,0,0.00,0.0000
1324360,21.8407,5.7226,0.00,0.0000,0.0000,0,0.00,38.1794
1324362,10.0000,0.0000,0.00,0.0000,0.0000,0,0.00,12.0000


In [9]:
# Reading User Daily Aggregation

daily <- read_sas("RawDataIIUserDailyAggregation.sas7bdat")
daily$ProductID <- mapvalues(daily$ProductID, from=c(1,2,3,4,5,6,7,8), to=c("Sports book fixed-odd","Sports book live-action","Poker BossMedia","Casino BossMedia","Supertoto","Games VS","Games bwin","Casino Chartwell"))
daily <- rename(daily,c('Product'='ProductID'))

daily$Date <- ymd(daily$Date)
daily <- daily %>% 
select(-Date) %>% group_by(UserID) %>% 
pivot_wider(names_from =Product,  values_from =c(Stakes, Winnings, Bets), values_fn = {sum})
daily[(is.na(daily))] <- 0
daily <- rename(daily,c('FOTotalStakes'='Stakes_Sports book fixed-odd'))
names(daily) <- c('UserID','FOTotalStakes','LATotalStakes','CasinoChartwellTotalStakes','VSTotalStakes','BwinTotalStakes', 'CasinoBossMediTotalStakes', 'SupertotoTotalStakes', 'FOTotalWinnings', 'LATotalWinnings', 'CasinoChartwellTotalWinnings', 'VSTotalWinnings', 'BwinTotalWinnings', 'CasinoBossMediaTotalWinnings', 'SupertotoTotalWinnings', 'FOTotalBets', 'LATotalBets', 'CasinoChartwellTotalBets', 'VSTotalBets', 'BwinTotalBets', 'CasinoBossMediaTotalBets', 'SupertotoTotalBets')


head(daily)
length(unique(daily$UserID))

The following `from` values were not present in `x`: 3


UserID,FOTotalStakes,LATotalStakes,CasinoChartwellTotalStakes,VSTotalStakes,BwinTotalStakes,CasinoBossMediTotalStakes,SupertotoTotalStakes,FOTotalWinnings,LATotalWinnings,...,BwinTotalWinnings,CasinoBossMediaTotalWinnings,SupertotoTotalWinnings,FOTotalBets,LATotalBets,CasinoChartwellTotalBets,VSTotalBets,BwinTotalBets,CasinoBossMediaTotalBets,SupertotoTotalBets
1324354,10137.3300,1839.2800,0,0,0,0,0,10224.1200,1512.4900,...,0,0,0,236,43,0,0,0,0,0
1324355,400.8600,24.7000,0,0,0,0,0,453.3000,11.2000,...,0,0,0,231,21,0,0,0,0,0
1324356,686.1900,679.0700,0,0,0,0,0,285.5100,625.1500,...,0,0,0,98,116,0,0,0,0,0
1324358,247.6971,88.5927,0,0,0,0,0,153.8756,55.9819,...,0,0,0,7,4,0,0,0,0,0
1324360,59.9993,1.7434,4,0,0,0,0,39.9564,1.2009,...,0,0,0,40,3,4,0,0,0,0
1324362,22.0000,0.0000,0,0,0,0,0,0.0000,0.0000,...,0,0,0,7,0,0,0,0,0,0


[1] 43851

In [10]:
# Checking column names

colnames(daily)

[1] "UserID"                       "FOTotalStakes"               
 [3] "LATotalStakes"                "CasinoChartwellTotalStakes"  
 [5] "VSTotalStakes"                "BwinTotalStakes"             
 [7] "CasinoBossMediTotalStakes"    "SupertotoTotalStakes"        
 [9] "FOTotalWinnings"              "LATotalWinnings"             
[11] "CasinoChartwellTotalWinnings" "VSTotalWinnings"             
[13] "BwinTotalWinnings"            "CasinoBossMediaTotalWinnings"
[15] "SupertotoTotalWinnings"       "FOTotalBets"                 
[17] "LATotalBets"                  "CasinoChartwellTotalBets"    
[19] "VSTotalBets"                  "BwinTotalBets"               
[21] "CasinoBossMediaTotalBets"     "SupertotoTotalBets"

In [11]:
# Creating SPdaily, GAdaily, CAdail & Categories

Spdaily <- daily %>% select(FOTotalStakes, LATotalStakes, FOTotalWinnings, LATotalWinnings, FOTotalBets, LATotalBets) %>%
group_by(UserID) %>%
transmute(SPbets=  FOTotalBets+ LATotalBets, SPstakes =FOTotalStakes+ LATotalStakes, SPwin= FOTotalWinnings+ LATotalWinnings)

GAdaily <- daily %>% select(BwinTotalStakes, SupertotoTotalStakes, BwinTotalBets, SupertotoTotalBets, BwinTotalWinnings, SupertotoTotalWinnings, VSTotalStakes, VSTotalWinnings, VSTotalBets) %>%
group_by(UserID) %>%
transmute(GAbets= BwinTotalBets+ SupertotoTotalBets +VSTotalBets, GAstakes =BwinTotalStakes+SupertotoTotalStakes+VSTotalStakes, GAwin= BwinTotalWinnings + SupertotoTotalWinnings + VSTotalWinnings)

CAdaily <- daily %>% select(CasinoChartwellTotalBets, CasinoBossMediaTotalBets, CasinoBossMediTotalStakes, CasinoChartwellTotalStakes, CasinoChartwellTotalWinnings, CasinoBossMediaTotalWinnings) %>%
group_by(UserID) %>%
transmute(CAbets= CasinoChartwellTotalBets +CasinoBossMediaTotalBets, CAstakes =CasinoBossMediTotalStakes+ CasinoChartwellTotalStakes, CAwin= CasinoChartwellTotalWinnings +CasinoBossMediaTotalWinnings)

categories <- dplyr::inner_join(Spdaily, GAdaily, by="UserID")
categories <- dplyr::inner_join(categories, CAdaily, by="UserID")

#head(CAdaily,3)
#head(GAdaily,3)
#head(Spdaily,3)
head(categories,3)

Adding missing grouping variables: `UserID`
Adding missing grouping variables: `UserID`
Adding missing grouping variables: `UserID`


UserID,SPbets,SPstakes,SPwin,GAbets,GAstakes,GAwin,CAbets,CAstakes,CAwin
1324354,279,11976.61,11736.61,0,0,0,0,0,0
1324355,252,425.56,464.50,0,0,0,0,0,0
1324356,214,1365.26,910.66,0,0,0,0,0,0


In [12]:
daily_first_date <- read_sas("RawDataIIUserDailyAggregation.sas7bdat")
daily_first_date$ProductID <- mapvalues(daily_first_date$ProductID, from=c(1,2,3,4,5,6,7,8), to=c('FirstSp','FirstSp','FirstPo','FirstCa','FirstGa','FirstGa','FirstGa','FirstCa'))
types_of_games <- unique(daily_first_date$ProductID)
daily_first_date <- daily_first_date %>% filter(Date <= 20050901)
daily_first_date$Date <- ymd(daily_first_date$Date)
daily_first_date <- daily_first_date %>% select(-Stakes,-Winnings, -Bets)
daily_first_date <- daily_first_date %>% group_by(UserID) %>%  pivot_wider(names_from = ProductID, values_from = Date, values_fn = {first})

daily_last_date <- read_sas("RawDataIIUserDailyAggregation.sas7bdat")
daily_last_date$ProductID <- mapvalues(daily_last_date$ProductID, from=c(1,2,3,4,5,6,7,8), to=c('LastSp','LastSp','LastPo','LastCa','LastGa','LastGa','LastGa','LastCa'))
types_of_games <- unique(daily_last_date$ProductID)
daily_last_date$Date <- ymd(daily_last_date$Date)
daily_last_date <- daily_last_date %>% select(-Stakes,-Winnings, -Bets)
daily_last_date <- daily_last_date %>% group_by(UserID) %>%  pivot_wider(names_from = ProductID, values_from = Date, values_fn = {last})

#merge
categories_dates <- dplyr::inner_join(daily_last_date, daily_first_date, by="UserID")

head(categories_dates,3)


The following `from` values were not present in `x`: 3
The following `from` values were not present in `x`: 3


UserID,LastSp,LastCa,LastGa,FirstSp,FirstCa,FirstGa
1324354,2005-09-30,NA,NA,2005-02-24,NA,NA
1324355,2005-09-29,NA,NA,2005-02-01,NA,NA
1324356,2005-09-12,NA,NA,2005-02-02,NA,NA


In [12]:
# Creating daily merged dataset

daily_merged <- dplyr::inner_join(categories, categories_dates, by="UserID")
daily_merged <- dplyr::inner_join(daily_merged, daily, by="UserID")
head(daily_merged)
length(unique(daily_merged$UserID))
length(daily_merged$UserID)

ERROR: Error in is.data.frame(y): object 'categories_dates' not found


In [13]:
# Reading Analytics Dataset

analytic <- read_sas("AnalyticDataInternetGambling.sas7bdat")
analytic$LANGUAGE <- analytic$LANGUAGE <- mapvalues(analytic$LANGUAGE,from=c(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22), to=c("English","German","Italian","Spanish","Swedish","French","Turkish","Greek","Polish","Norwegian","Danish","Catalan",
     "Czech","Hungarian","Dutch","Portugues","Russian","Slovenian","Croatian","Slovak","Simple Chinese","Traditional Chinese"))
analytic$GENDER <- mapvalues(analytic$GENDER, from=c(1,0), to=c('male', 'female'))
names(analytic) <- c('UserID', 'Country','Language','RegistrationDate','Age','Gender','FOTotalStakes','FOTotalWinnings','FOTotalBets','FOFirstActivedate','FOLastActivedate','FOTotalDaysActive','LATotalStakes','LATotalWinnings','LATotalBets','LAFirstActivedate','LALastActivedate','LATotalDaysActive','FirstSportsActivedate')

analytic <- analytic %>% select(-Country, -Language, -Gender)

#Find the min and max age to create Age_group
#analytic %>% select(Age) %>% summarize(max = max(Age), min = min(Age))

analytic <- analytic %>% mutate(Age_Group = case_when(Age >= 10  & Age <= 20 ~ '10-20',
                                                     Age >= 20  & Age <= 30 ~ '20-30',
                                                     Age >= 30  & Age <= 40 ~ '30-40',
                                                     Age >= 40  & Age <= 50 ~ '40-50',
                                                     Age >= 50  & Age <= 60 ~ '50-60',
                                                     Age >= 60  & Age <= 70 ~ '60-70',
                                                     Age >= 70  & Age <= 80 ~ '70-80',
                                                     Age >= 80  & Age <= 90 ~ '80-90',
                                                     Age > 90 ~ '90+'))
analytic$Age_Group[(is.na(analytic$Age_Group))] <- "Unknown"

analytic <- analytic %>% select(-FOTotalStakes, -FOTotalWinnings, -FOTotalBets)
head(analytic)
length(unique(analytic$UserID))
length(analytic$UserID)


The following `from` values were not present in `x`: 18, 19, 20, 21, 22


UserID,RegistrationDate,Age,FOFirstActivedate,FOLastActivedate,FOTotalDaysActive,LATotalStakes,LATotalWinnings,LATotalBets,LAFirstActivedate,LALastActivedate,LATotalDaysActive,FirstSportsActivedate,Age_Group
1324354,2005-02-01,42,2005-02-24,2005-09-30,97,1839.2800,1512.4900,43,2005-05-04,2005-09-26,18,2005-02-24,40-50
1324355,2005-02-01,22,2005-02-01,2005-09-29,72,24.7000,11.2000,21,2005-02-01,2005-02-07,7,2005-02-01,20-30
1324356,2005-02-01,28,2005-02-02,2005-09-12,38,679.0700,625.1500,116,2005-02-02,2005-09-11,23,2005-02-02,20-30
1324358,2005-02-01,24,2005-02-01,2005-05-06,5,88.5927,55.9819,4,2005-05-03,2005-05-03,1,2005-02-01,20-30
1324360,2005-02-01,27,2005-02-02,2005-09-25,17,1.7434,1.2009,3,2005-02-12,2005-09-21,2,2005-02-02,20-30
1324362,2005-02-01,22,2005-02-11,2005-09-17,5,NA,NA,NA,NA,NA,NA,2005-02-11,20-30


[1] 40499

[1] 40499

In [14]:
# Full join all the available datasets

database <- dplyr::full_join(daily_merged, analytic, by=c('UserID'))
database <- dplyr::full_join(database, poker_merged, by=c('UserID'))
database <- dplyr::full_join(database, demo, by=c('UserID'))
database <- dplyr::full_join(database, totalstakesmonth, by=c('UserID'))

database <- database %>% select(-FirstSp.y, -FirstGa.y, -FirstCa.x, -LATotalStakes.y, -RegistrationDate.x,
                                -LATotalWinnings.y,-LATotalBets.y,-FirstPo.y)

database <- rename(database,c('FirstSp'='FirstSp.x'))
database <- rename(database,c('FirstGa'='FirstGa.x'))
database <- rename(database,c('LATotalStakes'='LATotalStakes.x'))
database <- rename(database,c('LATotalWinnings'='LATotalWinnings.x'))
database <- rename(database,c('LATotalBets'='LATotalBets.x'))
database <- rename(database,c('FirstPo'='FirstPo.x'))
database <- rename(database,c('FirstCa'='FirstCa.y'))
database <- rename(database,c('RegistrationDate'='RegistrationDate.y'))


database$CountryName[(is.na(database$CountryName))] <- "Unknown"
database$Language[(is.na(database$Language))] <- "Unknown"
database$Gender[(is.na(database$Gender))] <- "Unknown"

length(unique(database$UserID))
#colSums(is.na(database))

ERROR: Error in dplyr::full_join(daily_merged, analytic, by = c("UserID")): object 'daily_merged' not found


In [15]:
# Replacing NA with Zero

database <- database %>% mutate_if(is.numeric, replace_na, replace = 0)

ERROR: Error in eval(lhs, parent, parent): object 'database' not found


In [16]:
# Checking Null Values

colSums(is.na(database))

ERROR: Error in is.data.frame(x): object 'database' not found


In [17]:
# Checking Column Names

colnames(database)

ERROR: Error in is.data.frame(x): object 'database' not found


In [18]:
# Plotting Gender Graph

options(repr.plot.width=6, repr.plot.height=4)

ggplot(database, aes(Gender, fill = Gender)) + 
    geom_bar()

ERROR: Error in ggplot(database, aes(Gender, fill = Gender)): object 'database' not found


In [19]:
# Checking Column Names

colnames(database)

ERROR: Error in is.data.frame(x): object 'database' not found


In [20]:
# Plotting Gender Vs CAWin

#Group_by Language Gender Group_Age ApplicationId
# Bet    POacive_dates GAbets CAbets Spbets
# win    GAwin CAwin POwinnings Spwin 
# Stakes PObets CAstakes GAstakes SPStakes

database %>%  
ggplot(aes(Gender, CAwin)) + geom_bar(stat='identity', na.rm=TRUE)

ERROR: Error in eval(lhs, parent, parent): object 'database' not found


In [21]:
# Creating French & Spain varaibles

fr <- database%>% group_by(CountryName) %>% filter(CountryName == 'France') %>% summarise(test = sum(CAbets))
esp <-  database%>% group_by(CountryName) %>% filter(CountryName == 'Spain') %>% summarise(test = sum(CAbets))

ERROR: Error in eval(lhs, parent, parent): object 'database' not found


In [22]:
# Summarizing CAbets for France & Spain

fr <- database%>% group_by(CountryName) %>% filter(CountryName == 'France') %>% summarise(test = sum(CAbets,na.rm=TRUE))
esp <-  database%>% group_by(CountryName) %>% filter(CountryName == 'Spain') %>% summarise(test = sum(CAbets,na.rm=TRUE))
df <- dplyr::full_join(fr, esp)
df

ggplot(df, aes(x=CountryName, y=test)) + 
    geom_bar(stat='identity', position='dodge')

ERROR: Error in eval(lhs, parent, parent): object 'database' not found


In [23]:
# Checking Column Names

colnames(database)

ERROR: Error in is.data.frame(x): object 'database' not found


In [24]:
# Formatting into Date Format

as.Date("20050930","%Y%m%d") - LastActiveDate[1]

ERROR: Error in eval(expr, envir, enclos): object 'LastActiveDate' not found


In [25]:
# Creating Test Variable

test <- head(database,100) %>% select(UserID, Age_Group,CountryName, LastSp, LastCa, LastGa, LastPo)
test$LastPo <- as.Date(test$LastPo,"%Y%m%d")

test <- test %>% mutate(recencyCa = mdy(09302005)- LastCa, recencyPo = mdy(09302005)- LastPo, recencyGa = mdy(09302005)- LastGa, recencySp = mdy(09302005)- LastSp) %>% select(-LastSp, -LastCa, -LastGa, -LastPo) 

test$recencyGa <- as.numeric(test$recencyGa)
test$recencyPo <- as.numeric(test$recencyPo)
test$recencyCa <- as.numeric(test$recencyCa)
test$recencySp <- as.numeric(test$recencySp)

head(test)
test <- test %>% filter(Age_Group =='20-30') %>% select(-Age_Group) %>%  group_by(CountryName) %>% 
summarise(median_recency_Ca = median(recencyCa, na.rm=TRUE), median_recency_Ga = median(recencyGa, na.rm=TRUE), median_recency_Sp = median(recencySp, na.rm=TRUE), median_recency_Po = median(recencyPo, na.rm=TRUE))

test <- test %>% tidyr::gather(month, "value", 2:5) %>% arrange(CountryName) %>%
     filter(CountryName =='France')

str(test)

ggplot(test, aes(x=month, y=value)) + geom_bar(stat='identity')

ERROR: Error in head(database, 100): object 'database' not found


In [26]:
# Checking Column Names

colnames(database)

ERROR: Error in is.data.frame(x): object 'database' not found


In [27]:
# Plotting Type Vs Number of Play

nbr_play <- database %>% select(UserID, Age_Group, CountryName, SPbets, CAbets, GAbets, PObets) %>% 
filter(Age_Group =='20-30') %>% select(-Age_Group) %>%  group_by(CountryName) %>% 
summarise(Sum_CA = sum(CAbets, na.rm=TRUE),Sum_GA = sum(GAbets, na.rm=TRUE), Sum_SP = sum(SPbets, na.rm=TRUE), Sum_PO = sum(PObets, na.rm=TRUE))

nbr_play <- nbr_play %>% tidyr::gather(type, "value", 2:5) %>% arrange(CountryName) %>%
     filter(CountryName =='France')



ggplot(nbr_play, aes(x=type, y=value)) + geom_bar(stat='identity')

ERROR: Error in eval(lhs, parent, parent): object 'database' not found


In [28]:
# Plotting Number of Players

count_player <- database %>% select(UserID, Age_Group, CountryName, SPbets, CAbets, GAbets, PObets) %>% 
filter(Age_Group =='20-30') %>% select(-Age_Group) %>%  group_by(CountryName) 
count_player[count_player ==0] <- NA

count_player <- count_player %>% summarise(Player_CA = sum(!is.na(CAbets)),Player_GA = sum(!is.na(GAbets)), Player_SP = sum(!is.na(SPbets)), Player_PO = sum(!is.na(PObets)))

head(count_player)

count_player <- count_player %>% tidyr::gather(type, "value", 2:5) %>% arrange(CountryName) %>%
     filter(CountryName =='France')


ggplot(count_player, aes(x=type, y=value)) + geom_bar(stat='identity')

ERROR: Error in eval(lhs, parent, parent): object 'database' not found


In [29]:
# Checking the head of the dataset

head(database,10) #%>% filter(CountryName == 'France')

ERROR: Error in head(database, 10): object 'database' not found


In [30]:
# Plotting Monthwise summary

monthplot <- database %>% select(UserID, Gender, CountryName, February, March, April, May, June, July, August, September)
monthplot <- monthplot %>% tidyr::gather(month, "value", 4:11) %>% 
    arrange(UserID) %>% 
    filter(CountryName =='France') %>%
    group_by(month) %>% 
    summarise(value = sum(value))%>% arrange(match(month,c("February", "March", "April", "May", "June", "July", "August", "September")) )

monthplot
#monthplot$month <- factor(monthplot$month, levels=c("February", "March", "April", "May", "June", "July", "August", "September"))

ggplot(monthplot, aes(x=month, y=value)) + geom_bar(stat='identity')

ERROR: Error in eval(lhs, parent, parent): object 'database' not found


In [31]:
# Plotting Agewise Summary

ggplot(database, aes(Age)) + geom_histogram(bins=25)

ERROR: Error in ggplot(database, aes(Age)): object 'database' not found


In [32]:
# Plotting Gender wise summary

df <- database %>% select(CountryName, Gender) %>% filter(CountryName == 'France') %>% group_by(CountryName, Gender) %>% summarise(test = n())
df

ggplot(df, aes(x=Gender, y= test)) + geom_bar(stat='identity')

ERROR: Error in eval(lhs, parent, parent): object 'database' not found


In [34]:
# Creating the final Marketing Data Mart

write.csv(database,"Rdatabase.csv")